<a href="https://colab.research.google.com/github/Syilun/AgeGenderDeepLearning/blob/master/cafeagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/GilLevi/AgeGenderDeepLearning.git

Cloning into 'AgeGenderDeepLearning'...
remote: Enumerating objects: 145, done.
remote: Total 145 (delta 0), reused 0 (delta 0), pack-reused 145
Receiving objects: 100% (145/145), 87.47 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [5]:
!sudo apt install libboost1.58-all-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libatlas-base-dev libgflags-dev libgoogle-glog-dev liblmdb-dev python-pip python-scipy python3-scipy -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libboost1.58-all-dev
E: Couldn't find any package by glob 'libboost1.58-all-dev'
E: Couldn't find any package by regex 'libboost1.58-all-dev'


In [6]:
!git clone https://github.com/BVLC/caffe.git

["Cloning into 'caffe'...",
 'remote: Enumerating objects: 65274, done.\x1b[K',
 'Receiving objects:   0% (1/65274)   ',
 'Receiving objects:   1% (653/65274)   ',
 'Receiving objects:   2% (1306/65274)   ',
 'Receiving objects:   3% (1959/65274)   ',
 'Receiving objects:   4% (2611/65274)   ',
 'Receiving objects:   5% (3264/65274)   ',
 'Receiving objects:   6% (3917/65274)   ',
 'Receiving objects:   7% (4570/65274)   ',
 'Receiving objects:   8% (5222/65274), 1.03 MiB | 1.97 MiB/s   ',
 'Receiving objects:   9% (5875/65274), 1.03 MiB | 1.97 MiB/s   ',
 'Receiving objects:  10% (6528/65274), 1.03 MiB | 1.97 MiB/s   ',
 'Receiving objects:  11% (7181/65274), 1.03 MiB | 1.97 MiB/s   ',
 'Receiving objects:  12% (7833/65274), 1.03 MiB | 1.97 MiB/s   ',
 'Receiving objects:  13% (8486/65274), 1.03 MiB | 1.97 MiB/s   ',
 'Receiving objects:  14% (9139/65274), 1.03 MiB | 1.97 MiB/s   ',
 'Receiving objects:  15% (9792/65274), 1.03 MiB | 1.97 MiB/s   ',
 'Receiving objects:  16% (10444/652

In [7]:
!apt install caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  caffe-tools-cuda cython3 fonts-lyx javascript-common libblosc1
  libcaffe-cuda1 libcublas9.1 libcudart9.1 libcurand9.1 libgflags2.2
  libgoogle-glog0v5 libjs-jquery libjs-jquery-ui libleveldb1v5 liblmdb0
  python-matplotlib-data python-tables-data python3-bs4 python3-caffe-cuda
  python3-chardet python3-cycler python3-dateutil python3-decorator
  python3-gflags python3-h5py python3-html5lib python3-ipython
  python3-ipython-genutils python3-leveldb python3-lxml python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-olefile python3-pandas
  python3-pandas-lib python3-pexpect python3-pickleshare python3-pil
  python3-pkg-resources python3-prompt-toolkit python3-protobuf
  python3-ptyproc

In [8]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL
from PIL import Image
import sys
import time

In [16]:
#定義Caffe根目錄
caffe_root = '/content/AgeGenderDeepLearning/'
#網路結構描述檔案
deploy_file_gender = caffe_root+'gender_net_definitions/deploy.prototxt'
deploy_file_age = caffe_root+'age_net_definitions/deploy.prototxt'
#訓練好的模型
model_age = caffe_root+'models/age_net.caffemodel'
model_gender = caffe_root+'models/gender_net.caffemodel'


In [20]:
import numpy as np
import sys
sys.path.append('/content/AgeGenderDeepLearning')			#caffe_root is your caffe root dir need change
import caffe

blob = caffe.proto.caffe_pb2.BlobProto()
data = open( '/content/AgeGenderDeepLearning/models/mean.binaryproto', 'rb' ).read()
blob.ParseFromString(data)
arr = np.array( caffe.io.blobproto_to_array(blob) )
out = arr[0]
np.save( 'mean' , out )

mean_file = '/content/mean.npy'

In [21]:
age_net = caffe.Classifier(
    deploy_file_age, # 呼叫deploy檔案
    model_age,  # 呼叫模型檔案
    mean=np.load(mean_file).mean(1).mean(1), # 呼叫均值檔案
    channel_swap=(2,1,0),  # caffe中圖片是BGR格式，而原始格式是RGB，所以要轉化
    raw_scale=255)         # python中將圖片儲存為[0, 1]，而caffe中將圖片儲存為[0, 255]，所以需要一個轉換

In [22]:
gender_net = caffe.Classifier(
    deploy_file_gender, # 呼叫deploy檔案
    model_gender,  # 呼叫模型檔案
    mean=np.load(mean_file).mean(1).mean(1), # 呼叫均值檔案
    channel_swap=(2,1,0),  # caffe中圖片是BGR格式，而原始格式是RGB，所以要轉化
    raw_scale=255)         # python中將圖片儲存為[0, 1]，而caffe中將圖片儲存為[0, 255]，所以需要一個轉換

In [ ]:
#分類標籤檔案
imagenet_labels_filename = caffe_root +'models/bvlc_googlenet/synset_words.txt'
#載入分類標籤檔案
labels = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')

In [ ]:
#對目標路徑中的影象，遍歷並分類
for root,dirs,files in os.walk(caffe_root+'models/bvlc_googlenet/image/'):
    for file in files:
        #載入要分類的圖片
        image_file = os.path.join(root,file)
        input_image = caffe.io.load_image(image_file)
        # print(type(input_image))
        #列印圖片路徑及名稱
        image_path = os.path.join(root,file)
        print(image_path)
        
        #顯示圖片
        img=Image.open(image_path)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
        
        #預測圖片類別
        time_start = time.time()
        prediction = net.predict([input_image])
        time_end = time.time() - time_start
        print 'Use time',time_end 
        print 'predicted class:',prediction[0].argmax()

        # 輸出概率最大的前5個預測結果
        top_k = prediction[0].argsort()[-5:][::-1]
        for node_id in top_k:     
            #獲取分類名稱
            human_string = labels[node_id]
            #獲取該分類的置信度
            score = prediction[0][node_id]
            print('%s (score = %.5f)' % (human_string, score))